# Exploring Ray API Calls

© 2019-2022, Anyscale. All Rights Reserved

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
➡ [Next notebook](./ex_07_ray_data.ipynb) <br>
⬅️ [Previous notebook](./ex_05_multiprocess_pool.ipynb) <br>


### Learning objectives
In this quick tour of the API, you will learn about:

 * Common Ray Core APIs
 * Some useful arguments to these APIs 
 * Tips and Tricks for first-time users


This lesson explores a few of the other API calls you might find useful, as well as options that can be used with the API calls we've already used in the previous lessons. Additionally, we will walk through some tips and tricks for first time users.

> **Tip:** The [Ray Package Reference](https://docs.ray.io/en/latest/package-ref.html) in the [Ray Docs](https://docs.ray.io/en/latest/) is useful for exploring the API features we'll learn.

In [1]:
import ray, time, sys, logging
import numpy as np 
import json

In [2]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.0.0rc1
Dashboard:,http://127.0.0.1:8265


## ray.init()

When we used [`ray.init()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.init), we used it to start Ray on our local machine. When the optional `address=...` argument is specified, the driver connects to the corresponding Ray cluster.

There are a lot of optional keyword arguments you can pass to `ray.init()`. Here are some of them. All options are described in the [documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.init). 

| Name | Type | Example | Description |
| :--- | :--- | :------ | :---------- |
| `address` | `str` | `address='auto'` | The address of the Ray cluster to connect to. If this address is not provided, then this command will start Redis, a raylet, a plasma store, a plasma manager, and some workers. It will also kill these processes when Python exits. If the driver is running on a node in a Ray cluster, using `auto` as the value tells the driver to detect the the cluster, removing the need to specify a specific node address. |
| `num_cpus` | `int` | `num_cpus=4` | Number of CPUs the user wishes to assign to each _raylet_. |
| `num_gpus` | `int` | `num_gpus=1` | Number of GPUs the user wishes to assign to each _raylet_. |
| `resources` | `dictionary` | `resources={'resource1': 4, 'resource2': 16}` | Maps the names of custom resources to the quantities of those resources available. |
| `memory` | `int` | `memory=1000000000` | The amount of memory (in bytes) that is available for use by workers requesting memory resources. By default, this is automatically set based on the available system memory. |
| `object_store_memory` | `int` | `object_store_memory=1000000000` | The amount of memory (in bytes) for the object store. By default, this is automatically set based on available system memory, subject to a 20GB cap. |
| `log_to_driver` | `bool` | `log_to_driver=True` | If true, then the output from all of the worker processes on all nodes will be directed to the driver program. |
| `local_mode` | `bool` | `local_mode=True` | If true, the code will be executed serially. This is useful for debugging. |
| `ignore_reinit_error` | `bool` | `ignore_reinit_error=True` | If true, Ray suppresses errors from calling `ray.init()` a second time (as we've done in these notebooks). Ray won't be restarted. |
| `configure_logging` | `bool` | `configure_logging=True` | If true (default), configuration of logging is allowed here. Otherwise, the user may want to configure it separately. |
| `logging_level` | _Flag_ | `logging_level=logging.INFO` | The logging level, defaults to `logging.INFO`. Ignored unless "configure_logging" is true. |
| `logging_format` | `str` | `logging_format='...'` | The logging format to use, defaults to a string containing a timestamp, filename, line number, and message. See the Ray source code `ray_constants.py` for details. Ignored unless "configure_logging" is true. |
| `runtime_env` | `map` | `{"working_dir": "/path/to/files"}` | Your Ray application might depend on source files or data files. For a development workflow, these might live on your local machine, but when it comes time to run things at scale, you will need to get them to your remote cluster. A way to send these files across all nodes in the cluster so that your distributed tasks or actors can access them, use this option, [for example.](https://docs.ray.io/en/latest/ray-core/handling-dependencies.html#runtime-environments) |

See also the documentation for [ray.shutdown()](https://ray.readthedocs.io/en/latest/package-ref.html#ray.shutdown), which is needed in some contexts.

## ray.is_initialized()

Is Ray [initialized](https://ray.readthedocs.io/en/latest/package-ref.html#ray.is_initialized)?

In [3]:
if not ray.is_initialized():
    ray.init()

## @ray.remote()

We've used [@ray.remote](https://ray.readthedocs.io/en/latest/package-ref.html#ray.remote) a lot. You can pass arguments when using it. Here are some of them.

| Name | Type | Example | Description |
| :--- | :--- | :------ | :---------- |
| `num_cpus` | `int` | `num_cpus=4` | The number of CPU cores to reserve for this task or for the lifetime of the actor. |
| `num_gpus` | `int` | `num_gpus=1` | The number of GPU cores to reserve for this task or for the lifetime of the actor. |
| `num_returns` | `int` | `num_returns=2` | (Only for tasks, not actors.) The number of object refs returned by the remote function invocation. |
| `runtime_env` | `map` | `runtime_env = {"working_dir": ".", "pip": ["requests"]}}` | The runtime environment to use for this job (see [Runtime environments](https://docs.ray.io/en/latest/ray-core/handling-dependencies.html#runtime-environments) for details. |
| `max_calls` | `int` | `max_calls=5` | Only for *remote tasks*. This specifies the maximum of times that a given worker can execute the given remote function before it must exit (this can be used to address memory leaks in third-party libraries or to reclaim resources that cannot easily be released, e.g., GPU memory that was acquired by TensorFlow). By default this is infinite. |
| `max_restarts` | `int` | `max_restarts=-1` | Only for *actors*. This specifies the maximum number of times that the actor should be restarted when it dies unexpectedly. The minimum valid value is 0 (default), which indicates that the actor doesn't need to be restarted. A value of -1 indicates that an actor should be restarted indefinitely. |
| `max_task_retries` | `int` | `max_task_retries=-1` | Only for *actors*. How many times to retry an actor task if the task fails due to a system error, e.g., the actor has died. If set to -1, the system will retry the failed task until the task succeeds, or the actor has reached its max_restarts limit. If set to n > 0, the system will retry the failed task up to n times, after which the task will throw a `RayActorError` exception upon `ray.get`. Note that Python exceptions are not considered system errors and will not trigger retries. |
| `max_retries` | `int` | `max_retries=-1` | Only for *remote functions*. This specifies the maximum number of times that the remote function should be rerun when the worker process executing it crashes unexpectedly. The minimum valid value is 0, the default is 4 (default), and a value of -1 indicates infinite retries. |

Here's an example with and without `num_return_vals`:

In [7]:
@ray.remote(num_returns=3)
def tuple3(one, two, three):
    return (one, two, three)

# return three object references with three distinct values in each 
x_ref, y_ref, z_ref = tuple3.remote("a", 1, 2.2)
x, y, z = ray.get([x_ref, y_ref, z_ref])
print(f'({x}, {y}, {z})')

(a, 1, 2.2)


In [8]:
@ray.remote(num_returns=1)
def tuple3(one, two, three):
    return (one, two, three)

# returns one object references with three values in it
xyz_ref = tuple3.remote("a", 1, 2.2)
x, y, z = ray.get(xyz_ref)
print(f'({x}, {y}, {z})')

(a, 1, 2.2)


### @ray.method()

Related to `@ray.remote()`, [@ray.method()](https://ray.readthedocs.io/en/latest/package-ref.html#ray.method) allows you to specify the number of return values for a method in an actor, by passing the `num_returns` keyword argument. None of the other `@ray.remote()` keyword arguments are allowed. Here is an example:

In [6]:
@ray.remote
class Tupleator:
    @ray.method(num_returns=3)
    def tuple3(self, one, two, three):
        return (one, two, three)
    
tupleator = Tupleator.remote()
x_ref, y_ref, z_ref = tupleator.tuple3.remote("a", 1, 2.2)
x, y, z = ray.get([x_ref, y_ref, z_ref])
print(f'({x}, {y}, {z})')   

(a, 1, 2.2)


## ray.put()

We used [`ray.get`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.gett) a lot to retrieve objects and we used actor methods to retrieve state from an actor. You can actually put objects into the object store explicitly with [`ray.put`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.put), as shown in the following example:

In [6]:
ref = ray.put("Hello World!")
print(f'Object returned: {ray.get(ref)}')

Object returned: Hello World!


In [7]:
ref = ray.put(np.random.rand(2_000, 5_000))
print(f'Object returned: {ray.get(ref)}')

Object returned: [[0.27417081 0.91811842 0.9755556  ... 0.07707923 0.11121964 0.09837812]
 [0.34833991 0.26149561 0.2538808  ... 0.89033091 0.97065292 0.47511148]
 [0.25735175 0.2953164  0.87389208 ... 0.27095118 0.88620768 0.94448367]
 ...
 [0.29690232 0.75853874 0.10129418 ... 0.25739422 0.66575173 0.43192699]
 [0.29109451 0.95571804 0.33055667 ... 0.11788663 0.57839189 0.27331154]
 [0.4862909  0.067976   0.40543115 ... 0.27321733 0.29416929 0.05094307]]


There is an optional flag you can pass `weakref=True` (defaults to `False`). If true, Ray is allowed to evict the object while a reference to the returned ref still exists. This is useful if you are putting a lot of objects into the object store and many of them might not be needed in the future. It allows Ray to aggressively reclaim memory.

## Fetching Cluster Information

Many methods return information:

| Method | Brief Description |
| :----- | :---------------- |
| [`ray.get_gpu_ids()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.get_gpu_ids) | GPUs |
| [`ray.nodes()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.nodes) | Cluster nodes |
| [`ray.cluster_resources()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.cluster_resources) | All the available resources, used or not |
| [`ray.available_resources()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.available_resources) | Resources not in use |

In [5]:
print(f"""
ray.get_gpu_ids():          {ray.get_gpu_ids()}
ray.nodes():                {ray.nodes()}
ray.cluster_resources():    {ray.cluster_resources()}
ray.available_resources():  {ray.available_resources()}
""")


ray.get_gpu_ids():          []
ray.nodes():                [{'NodeID': '24237b52c668315145482a3fd00efcbbc72ac9e91208f2f0a283b7cb', 'Alive': True, 'NodeManagerAddress': '127.0.0.1', 'NodeManagerHostname': 'Juless-MacBook-Pro-16', 'NodeManagerPort': 63908, 'ObjectManagerPort': 63907, 'ObjectStoreSocketName': '/tmp/ray/session_2022-08-11_08-47-22_387133_2819/sockets/plasma_store', 'RayletSocketName': '/tmp/ray/session_2022-08-11_08-47-22_387133_2819/sockets/raylet', 'MetricsExportPort': 64235, 'NodeName': '127.0.0.1', 'alive': True, 'Resources': {'memory': 42834706432.0, 'CPU': 10.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 2147483648.0}}]
ray.cluster_resources():    {'memory': 42834706432.0, 'object_store_memory': 2147483648.0, 'CPU': 10.0, 'node:127.0.0.1': 1.0}
ray.available_resources():  {'memory': 42834706432.0, 'object_store_memory': 2147483648.0, 'CPU': 10.0, 'node:127.0.0.1': 1.0}



Recall that we used `ray.nodes()[0]['Resources']['CPU']` in the second lesson to determine the number of CPU cores on our machines:

In [6]:
ray.nodes()[0]['Resources']['CPU']

10.0

# Tips and Tricks for first-time users
First time users can trip upon certain API calls in the usage patterns. This short tip & triks will insure you against unexpected results. Below we briefly explore of a handful of API calls and their best practice.

### Tip 1: Delay ray.get()

With Ray, all invocations of `.remote()` calls are asynchronous, meaning the operation  returns immediately with a promise/future object ID. This is key to achieving massive parallelism, for it allows a devloper to launch many remote tasks, each returning a remote future object ID. Whenever needed, this object ID is fetched with `ray.get`. Because `ray.get` is a blocking call, where and how often you use affects the performance. 


In [7]:
@ray.remote
def do_some_work(x):
    time.sleep(1)
    return x * x

#### Bad usage
We use `ray.get` inside a loop since it blocks on each call of `.remote()`

In [8]:
%%time
results = [ray.get(do_some_work.remote(x)) for x in range(10)]
results

(tuple3 pid=2933) E0811 08:51:08.351647000 6201012224 chttp2_transport.cc:1111]          Received a GOAWAY with error code ENHANCE_YOUR_CALM and debug data equal to "too_many_pings"


CPU times: user 74.9 ms, sys: 39.7 ms, total: 115 ms
Wall time: 11.1 s


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

#### Good usage
We delay `ray.get` after all the tasks have been invoked and their references have been returned.


In [9]:
%%time
results = ray.get([do_some_work.remote(x) for x in range(10)])
results

CPU times: user 17.9 ms, sys: 10.7 ms, total: 28.6 ms
Wall time: 1.5 s


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

#### Takeway tip 1: 
Since `ray.get` is a blocking call, postpone its use only when you need object ID's value. If called eagerly, it can
affect the performance of your desired parallelism.

### Tip 2: Avoid tiny remote tasks
Ray APIs are general and simple to use. As a result, new comers' natural intinct is to parallelize all tasks, including tiny ones, which can incur an overhead overtime. 
In short, if the Ray remote tasks are tiny or miniscule in compute, they may take longer to execute than their serial Python equivalents.

In [10]:
def tiny_task(x):
    time.sleep(0.0001)
    return x

In [11]:
%%time
results = [tiny_task(x) for x in range(100000)]
results[:10]

CPU times: user 107 ms, sys: 161 ms, total: 269 ms
Wall time: 12.9 s


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Now convert this into Ray remote task

In [12]:
@ray.remote
def remote_tiny_task(x):
    time.sleep(0.0001)
    return x

In [13]:
%%time
result_ids = [remote_tiny_task.remote(x) for x in range(100000)]
results = ray.get(result_ids)
results[:10]

(do_some_work pid=3337) E0811 08:54:52.702333000 6142193664 chttp2_transport.cc:1111]          Received a GOAWAY with error code ENHANCE_YOUR_CALM and debug data equal to "too_many_pings"


CPU times: user 6.9 s, sys: 2.51 s, total: 9.41 s
Wall time: 6.12 s


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Surprisingly, not only Ray didn’t improve the execution time, but the Ray program is actually slower than the sequential program! What can we do to remedy it? What's going on?

Well, the issue here is that every task invocation has a non-trivial overhead (e.g., scheduling, inter-process communication, updating the system state) and this overhead dominates the actual time it takes to execute the task.

One way to mitigate is to make the remote tasks "larger" in order to amortize invocation overhead. This is achieved by aggregating tasks into bigger chunks of 1000.


In [14]:
@ray.remote
def mega_work(start, end):
    return [tiny_task(x) for x in range(start, end)]

In [15]:
%%time
result_ids = []
[result_ids.append(mega_work.remote(x*1000, (x+1)*1000)) for x in range(100)]
results = ray.get(result_ids)

CPU times: user 204 ms, sys: 24 ms, total: 228 ms
Wall time: 1.65 s


A huge difference in execution time!

### Tip 3: Using ray.wait() with ray.get()

As we noted above, an idiomatic way of using `ray.get()` is delay fetching the object until you need them. Another way is to use is with `ray.wait()`, and only fetch values that are already available. 

Let's look at a simple example.

In [16]:
import numpy as np
@ray.remote
def make_array(n):
    time.sleep(n/10.0)
    return np.random.standard_normal(n)

Now define a task that can add two NumPy arrays together. The arrays need to be the same size, but we'll ignore any checking for this requirement.

In [17]:
@ray.remote
def add_arrays(a1, a2):
    time.sleep(a1.size/10.0)
    return np.add(a1, a2)

Now let's use `ray.wait` and `ray.get`

In [18]:
%%time

array_refs = [make_array.remote(n*10) for n in range(6)]
added_array_refs = [add_arrays.remote(ref, ref) for ref in array_refs]

arrays = []
waiting_refs = list(added_array_refs)  # Assign a working list to the full list of refs
while len(waiting_refs) > 0:           # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of refs we're still waiting to complete,
    #   2. tell it to return immediately as soon as one of them completes,
    #   3. tell it wait up to 10 seconds before timing out.
    ready_refs, remaining_refs = ray.wait(waiting_refs, num_returns=2, timeout=10.0)
    new_arrays = ray.get(ready_refs)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_refs = remaining_refs  # Reset this list; don't include the completed refs in the list again!
    
# print(f"\nall arrays: {arrays}")

0: []
10: [-2.6273131  -3.1779646  -4.64840253 -4.17354143 -2.4364305   0.70570382
  2.518578   -2.25959921  0.06402084 -0.44575623]
20: [ 0.24589101  1.7950238  -2.62617243  0.39362694  0.92088594  3.62210851
  1.97069318 -1.03811819  3.13761216 -4.39574758  1.38489855 -0.27061955
 -3.43540906  0.62856759  1.99349043  3.5674094   1.65869368  2.79011352
 -1.30261914 -0.48447068]
30: [ 1.77418785  6.656585   -2.6446964  -0.16805503  0.42810247  2.42533177
  2.02960033  2.00254389 -1.18141709  1.23375361 -0.8669101   4.15729229
  2.1220753   3.95442161 -0.8227272  -5.80612409  1.71607275 -3.48392167
  2.6459461  -0.85549549 -1.89468767  3.3399131  -0.51754745 -2.66212115
 -0.57902311  0.25142483  0.23297274  1.55461581 -0.2132207   1.95231483]
40: [-2.16061672 -0.71083383 -0.4681008  -1.78180955 -2.01400651 -0.49849118
  0.45590618 -2.10168433 -2.48517716 -0.07551191  1.60982934  2.17100561
  0.31040109 -4.6825149  -1.63700511 -2.64691206  0.01588598  0.13583479
  1.79142911 -0.11396917 

In [23]:
ray.shutdown()

### Next Step

Let's move on to our final module 3 and start with [Ray Datasets lesson](ex_07_ray_data.ipynb)

### Homework 

Read some more [tricks and tips](https://docs.ray.io/en/latest/ray-core/tips-for-first-time.html) in the documentation

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
➡ [Next notebook](./ex_07_ray_data.ipynb) <br>
⬅️ [Previous notebook](./ex_05_multiprocess_pool.ipynb) <br>